In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [3]:
quote_text = pd.read_csv('GNI88.csv')

In [4]:
article_data = pd.read_json("gni88.json", lines=True)
article_data.head()

,Article ID,Headline,Content,Media Name,Author,Published Date
0,3777409,Modernized Tu-160 to boost Russia's long-range...,Modernized Tu-160 to boost Russia's long-range...,Defense News,"Bodner, Matthew",2018-02-15 00:00:00+00
1,3764250,Give Trump more nuclear weapons and more ways ...,Give Trump more nuclear weapons and more ways ...,CNN,"Collina, Tom Z.",2018-02-02 00:00:00+00
2,3764251,Trump wants to build smaller atomic weapons; R...,Trump wants to build smaller atomic weapons; R...,Los Angeles Times,"Cloud, David",2018-02-03 00:00:00+00
3,3764252,Egypt and Israel Secretly Allied In Sinai Battle,Egypt and Israel Secretly Allied In Sinai Batt...,The New York Times,"Kirkpatrick, David D.",2018-02-04 00:00:00+00
4,3764253,Listen: Top Armed Services Dem worried about r...,Listen: Top Armed Services Dem worried about r...,The Hill,"Simendinger, Alexis",2018-02-03 00:00:00+00


In [15]:
len(article_data)

69995

# Extract first 10% of article content

make new content column -- does not include title, author, date

In [29]:
# only take everything after date in content
def justContent(content):
    content = ''.join(content.split('\r\n\r\n')[2:])
    return content

In [49]:
article_data['Content2'] = article_data['Content'].apply(lambda x: justContent(x))

In [51]:
article_data['First_10%_Content_Split'] = article_data['Content2'].apply(lambda x: x.split()[:int(0.1*len(x))])

In [52]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
article_data['First_10%_Content'] = article_data['First_10%_Content_Split'].apply(lambda x: TreebankWordDetokenizer().detokenize(x))

In [53]:
# takes only columns with content
df = article_data.loc[article_data['First_10%_Content_Split'].apply(lambda x: len(x) >0)]

In [54]:
merged_df = quote_text.merge(df[['Article ID', 'Content', 'Author', 'Published Date', 'First_10%_Content', 'First_10%_Content_Split']], on = 'Article ID')

# Countries, people, treaties that could point to messages/topics

In [57]:
merged_df['First_10%_Content_Split']

0         [SEOUL, -, North, Korea's, representatives, as...
1         [SEOUL, -, North, Korea's, representatives, as...
2         [SEOUL, -, North, Korea's, representatives, as...
3         [SEOUL, -, North, Korea's, representatives, as...
4         [SEOUL, -, North, Korea's, representatives, as...
                                ...                        
301268    [But, was, she, celebrated, for, it?, Not, so,...
301269    [But, was, she, celebrated, for, it?, Not, so,...
301270    [On, the, one, hand,, this, was, hilarious,, s...
301271    [On, the, one, hand,, this, was, hilarious,, s...
301272    [On, the, one, hand,, this, was, hilarious,, s...
Name: First_10%_Content_Split, Length: 301273, dtype: object

# extract treaties
creating a column with treaties mentioned in the 1st 10% content
- for each 1st 10% content (split), extract any mentions of treaties (take the capital words around it too)

In [63]:
import regex as re

In [108]:
# function that returns the 5 words before the word "Treaty"
def findTreaty(content):
    pattern = r"((?:\S+\s+){0,5}\bTreaty\b*(?:\S+\b\s*){0,0})"
    return re.findall(pattern, content)


In [109]:
# applies findTreaty to 1st 10% content
merged_df['Treaty_Instances'] = merged_df['First_10%_Content'].apply(lambda x: findTreaty(x))

Given each row in Treaty_Instances --> starting from Treaty, works backwards and returns everything until reaches non-proper word



In [133]:
# extract everything from first mention of "the" to Treaty
# treaty_instance is each item in merged_df['Treaty_Instances']
def propWords_Treaty(treaty_instance):
    if treaty_instance == []:
        return None
    words = treaty_instance[0].split()
    for i in np.arange(len(words)):
        if words[i] == 'the':
            return ' '.join(words[i:])

In [122]:
merged_df['Treaty_Instances'].value_counts().index[5]

['signatory of the Nuclear Non-Proliferation Treaty']

In [130]:
propWords_Treaty(merged_df['Treaty_Instances'].value_counts().index[5])

'the Nuclear Non-Proliferation Treaty'

In [134]:
# applies propWords_Treaty to Treaty_Instances
merged_df['Treaties'] = merged_df['Treaty_Instances'].apply(lambda x: propWords_Treaty(x))

In [137]:
merged_df['Treaties'].value_counts()

the Nuclear Nonproliferation Treaty                1507
the Nuclear Non-Proliferation Treaty               1365
the nuclear Non-Proliferation Treaty                790
the North Atlantic Treaty                           693
the Non-Proliferation Treaty                        683
                                                   ... 
the Arms Trade Treaty                                 1
the 1963 Franco-German Elysée Treaty                  1
the long run."The Outer Space Treaty                  1
the Russian exercises.The North Atlantic Treaty       1
the 1967 U.N. Outer Space Treaty                      1
Name: Treaties, Length: 235, dtype: int64

In [152]:
#problem: if there are more than one instance of "the", it includes it
# try going from "Treaty" backwards, so it finds the last "the" first
def propWords_Treaty2(treaty_instance):
    if treaty_instance == []:
        return None
    words = treaty_instance[0].split()
    i = len(words)-1
    while i > 0:#<= (len(words)-1):
        if words[i] == 'the':
            return ' '.join(words[i:])
        else:
            i -= 1
        

In [153]:
merged_df['Treaty_Instances'].value_counts().index[5]

['signatory of the Nuclear Non-Proliferation Treaty']

In [154]:
propWords_Treaty2(merged_df['Treaty_Instances'].value_counts().index[5])

'the Nuclear Non-Proliferation Treaty'

In [155]:
# applies propWords_Treaty to Treaty_Instances
merged_df['Treaties2'] = merged_df['Treaty_Instances'].apply(lambda x: propWords_Treaty2(x))

In [156]:
merged_df['Treaties2'].value_counts()

the Nuclear Nonproliferation Treaty              1571
the Nuclear Non-Proliferation Treaty             1365
the nuclear Non-Proliferation Treaty              790
the Non-Proliferation Treaty                      716
the North Atlantic Treaty                         704
                                                 ... 
the organization's founding Washington Treaty       1
the Iran Nuclear Treaty                             1
the Oslo Treaty                                     1
the Arms Trade Treaty                               1
the summit, North Atlantic Treaty                   1
Name: Treaties2, Length: 163, dtype: int64

Still some issues with some of the strings:

* "the organization's founding Washington Treaty"
* "the summit, North Atlantic Treaty"

not all the treaties start with "the"
future work: improve Treaty extraction

# Bag of Words Model

basic text cleaning

In [159]:
text_lowercase = merged_df['First_10%_Content'].str.lower()
text_lowercase

0         seoul - north korea's representatives assured ...
1         seoul - north korea's representatives assured ...
2         seoul - north korea's representatives assured ...
3         seoul - north korea's representatives assured ...
4         seoul - north korea's representatives assured ...
                                ...                        
301268    but was she celebrated for it? not so much. - ...
301269    but was she celebrated for it? not so much. - ...
301270    on the one hand, this was hilarious, since on ...
301271    on the one hand, this was hilarious, since on ...
301272    on the one hand, this was hilarious, since on ...
Name: First_10%_Content, Length: 301273, dtype: object

remove punctuation

In [160]:
from string import punctuation

def remove_punctuation(document):
    no_punct = ''.join([character for character in document if character not in punctuation])
    return no_punct

In [161]:
text_no_punct = text_lowercase.apply(remove_punctuation)
text_no_punct

0         seoul  north koreas representatives assured th...
1         seoul  north koreas representatives assured th...
2         seoul  north koreas representatives assured th...
3         seoul  north koreas representatives assured th...
4         seoul  north koreas representatives assured th...
                                ...                        
301268    but was she celebrated for it not so much  mei...
301269    but was she celebrated for it not so much  mei...
301270    on the one hand this was hilarious since on mo...
301271    on the one hand this was hilarious since on mo...
301272    on the one hand this was hilarious since on mo...
Name: First_10%_Content, Length: 301273, dtype: object

remove digits

In [162]:
def remove_digit(document): 
    no_digit = ''.join([character for character in document if not character.isdigit()]) 
    return no_digit

In [163]:
text_no_digit = text_no_punct.apply(remove_digit)
text_no_digit

0         seoul  north koreas representatives assured th...
1         seoul  north koreas representatives assured th...
2         seoul  north koreas representatives assured th...
3         seoul  north koreas representatives assured th...
4         seoul  north koreas representatives assured th...
                                ...                        
301268    but was she celebrated for it not so much  mei...
301269    but was she celebrated for it not so much  mei...
301270    on the one hand this was hilarious since on mo...
301271    on the one hand this was hilarious since on mo...
301272    on the one hand this was hilarious since on mo...
Name: First_10%_Content, Length: 301273, dtype: object

TOKENIZATION

transform text string into vector of words

In [164]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/atmika/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [165]:
from nltk.tokenize import word_tokenize

text_tokenized = text_no_digit.apply(word_tokenize)
text_tokenized.head()

0    [seoul, north, koreas, representatives, assure...
1    [seoul, north, koreas, representatives, assure...
2    [seoul, north, koreas, representatives, assure...
3    [seoul, north, koreas, representatives, assure...
4    [seoul, north, koreas, representatives, assure...
Name: First_10%_Content, dtype: object

remove stopwords

In [166]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/atmika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [167]:
def remove_stopwords(document):
    words = [word for word in document if not word in stop_words]
    return words

In [168]:
text_no_stop = text_tokenized.apply(remove_stopwords)
text_no_stop

0         [seoul, north, koreas, representatives, assure...
1         [seoul, north, koreas, representatives, assure...
2         [seoul, north, koreas, representatives, assure...
3         [seoul, north, koreas, representatives, assure...
4         [seoul, north, koreas, representatives, assure...
                                ...                        
301268    [celebrated, much, meitner, collaborated, seve...
301269    [celebrated, much, meitner, collaborated, seve...
301270    [one, hand, hilarious, since, monday, dow, sec...
301271    [one, hand, hilarious, since, monday, dow, sec...
301272    [one, hand, hilarious, since, monday, dow, sec...
Name: First_10%_Content, Length: 301273, dtype: object

STEMMING

In [169]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
def stemmer(document):
    stemmed_document = [porter.stem(word) for word in document]
    return stemmed_document

In [170]:
text_stemmed = text_no_stop.apply(stemmer)
text_stemmed

0         [seoul, north, korea, repres, assur, south, ko...
1         [seoul, north, korea, repres, assur, south, ko...
2         [seoul, north, korea, repres, assur, south, ko...
3         [seoul, north, korea, repres, assur, south, ko...
4         [seoul, north, korea, repres, assur, south, ko...
                                ...                        
301268    [celebr, much, meitner, collabor, sever, decad...
301269    [celebr, much, meitner, collabor, sever, decad...
301270    [one, hand, hilari, sinc, monday, dow, secondw...
301271    [one, hand, hilari, sinc, monday, dow, secondw...
301272    [one, hand, hilari, sinc, monday, dow, secondw...
Name: First_10%_Content, Length: 301273, dtype: object

# Document-Term Matrix
calculate frequency of words across news articles

detokenization

In [171]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_detokenized = text_stemmed.apply(TreebankWordDetokenizer().detokenize)

document-term matrix

In [172]:
from sklearn.feature_extraction.text import CountVectorizer

countvec = CountVectorizer()

sparse_dtm = countvec.fit_transform(text_detokenized)
sparse_dtm

<301273x419526 sparse matrix of type '<class 'numpy.int64'>'
	with 77216234 stored elements in Compressed Sparse Row format>

In [1]:
dtm = pd.DataFrame(sparse_dtm.toarray(), columns=countvec.get_feature_names(), index=merged_df.index)
dtm

NameError: name 'pd' is not defined

In [ ]:
frequencies = dtm.sum().sort_values(ascending=False)
print(frequencies[frequencies > 50])

future work: figure out why kernel is dying, find most common word features

In [ ]:
# plot to visualize frequency of words
plt.figure(figsize=(8,6))

ax = sns.countplot(frequencies)
plt.xticks(np.arange(1, 50, step=5), np.arange(1, 50, step=5))


plt.xlabel('terms')
plt.ylabel(' ')
plt.show()